In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pathlib
import xarray as xr

In [ ]:
OBS_DIR = '***BASEDIR***training-output-reference/'
obs_path = pathlib.Path(OBS_DIR)

obs_files = sorted([f for f in obs_path.iterdir() if 'tp' in f.stem])

In [ ]:
def fix_dataset_dims(d):
    """Given one of the dataset files given by the organizers, fix its
    dimensions so its easier to concatenate and use with xr.open_mfdataset.

    Arguments:
      d. xr.Dataset. The dataset you get when you open one of the provided files.
    """

    day_of_year = d.forecast_time[0].dt.dayofyear.data.item()

    new_d = d.expand_dims("forecast_dayofyear").assign_coords(
        forecast_dayofyear=[day_of_year]
    )
    new_d = new_d.assign_coords(forecast_year=new_d.forecast_time.dt.year).swap_dims(
        forecast_time="forecast_year"
    )

    # Reorder the dimensions to something that is more intuitive (according to me).
    dims = set(new_d.dims)
    dims.difference_update(
        ("forecast_dayofyear", "forecast_year", "latitude", "longitude")
    )

    new_d = new_d.transpose(
        "forecast_year", "forecast_dayofyear", *dims, "latitude", "longitude"
    )

    # new_d = new_d.chunk(chunks="auto")

    return new_d


In [ ]:
tp = xr.open_mfdataset(obs_files, preprocess=fix_dataset_dims)

In [ ]:
tp

In [ ]:
tp.sel(forecast_dayofyear=86, forecast_year=2015).isnull().sum(dim=['latitude', 'longitude']).tp.compute().plot()

In [ ]:
lead1 = tp.sel(forecast_dayofyear=86, forecast_year=2015).isel(lead_time=1).isnull().tp.compute().astype('uint')

In [ ]:
tp

In [ ]:
(tp.sel(forecast_dayofyear=86, forecast_year=2015).isel(lead_time=0).isnull().sum(dim='lead_time') - tp.sel(forecast_dayofyear=86, forecast_year=2015).isel(lead_time=1).isnull().sum(dim='lead_time')).tp.compute()

In [ ]:
tp.forecast_dayofyear

In [ ]:
tp.sel(forecast_dayofyear=51, forecast_year=2012).isnull().sum(dim='lead_time').compute().tp.plot()

In [ ]:
tp.sel??